# Assignment 3

First import all the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

Load datasets?

In [3]:
df_houseprices_testdata = pd.read_csv('test.csv', sep=',')
df_houseprices_traindata = pd.read_csv('train.csv', sep=',')

Look at the data

In [5]:
df_houseprices_testdata.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
df_houseprices_traindata.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Now I need to figure out what to focus on, I might want the fields:
    - (YearRemodAdd - YrSold)*-1 = delta_years_mod_and_sold
    - LotArea OK
    - OverallCond OK
    - OverallQual OK
    - MSZoning OK

In [44]:
df_houseprices_traindata["MSZoning"].describe()

count     1460
unique       5
top         RL
freq      1151
Name: MSZoning, dtype: object

It is obvious that there is many low density residential zoned houses, I'll now separate the different MSZones to their own coloumn

In [66]:
MSZones_seperated = pd.get_dummies(df_houseprices_traindata["MSZoning"], prefix="MSZone", drop_first=True, dtype=int)
MSZones_seperated.head()

,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


Now that we have separated them into coloumns, we can select the other Quality, condition and LotArea coloumns. 
Then we join them together, we now just need the last coloumn; "delta_years_mod_and_sold"

In [112]:
df_Choosen_numeric_DataColoumns = pd.DataFrame(df_houseprices_traindata[["OverallQual","LotArea","OverallCond"]])
df_all_but_yearDelta_coloumns = df_Choosen_numeric_DataColoumns.join(MSZones_seperated)
df_all_but_yearDelta_coloumns.head()

,OverallQual,LotArea,OverallCond,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM
0,7,8450,5,0,0,1,0
1,6,9600,8,0,0,1,0
2,7,11250,5,0,0,1,0
3,7,9550,5,0,0,1,0
4,8,14260,5,0,0,1,0


We calculate the delta_years_mod_and_sold like so: (YearRemodAdd - YrSold)*-1

In [115]:
yearRemodAdd = df_houseprices_traindata["YearRemodAdd"]
yearsold = df_houseprices_traindata["YrSold"]
calculatedCols = (yearRemodAdd-yearsold) * -1
delta_years_mod_and_sold = pd.DataFrame(calculatedCols,columns=["delta_years_mod_and_sold"])
delta_years_mod_and_sold.head()

,delta_years_mod_and_sold
0,5
1,31
2,6
3,36
4,8


In [119]:
finaldataset = df_all_but_yearDelta_coloumns.join(delta_years_mod_and_sold)
finaldataset

,OverallQual,LotArea,OverallCond,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM,delta_years_mod_and_sold
0,7,8450,5,0,0,1,0,5
1,6,9600,8,0,0,1,0,31
2,7,11250,5,0,0,1,0,6
3,7,9550,5,0,0,1,0,36
4,8,14260,5,0,0,1,0,8
5,5,14115,5,0,0,1,0,14
6,8,10084,5,0,0,1,0,2
7,7,10382,6,0,0,1,0,36
8,7,6120,5,0,0,0,1,58
9,5,7420,6,0,0,1,0,58
